# Caso Práctico - Visualización de Datos
---
Visualización de Datos del Máster Universitario en Ingeniería y Ciencia de Datos (2023/2024)

Alumno: **Javier Orive Soto**


# 1. Importación de dependencias

In [89]:
import pandas as pd
import panel as pn
import numpy as np
import altair as alt

import datetime as dt

# 2. Importación de datos

## 2.1 Datos económicos

In [65]:
# Cargar el archivo Excel
#file_path = 'Datos/Economico/PrecioMedioHorarioFinal_2023_d.xls'
file_path = 'Datos/Generacion/GeneracionTotal_2023_h.csv'
#df = pd.read_excel(file_path)
df = pd.read_csv(file_path, delimiter=';')

# Seleccionar solo las columnas de interés
#df = df[['name', 'value', 'datetime']]

In [66]:
# Convertir la columna datetime a tipo datetime
df['datetime'] = pd.to_datetime(df['datetime'], utc = True) #, utc = True

# Convertir la columna value a tipo float
df['value'] = df['value'].astype(float)

# Convertir la columna name a tipo categoría
df['name'] = df['name'].astype('category')


In [67]:
df = df.drop(['geoid', 'geoname'], axis=1)

In [68]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour

In [69]:
df.head()

,id,name,value,datetime,year,month,day,hour
0,10195,Generación medida total tipo producción,19130.484,2022-12-31 23:00:00+00:00,2022,12,31,23
1,10195,Generación medida total tipo producción,19230.209,2023-01-01 00:00:00+00:00,2023,1,1,0
2,10195,Generación medida total tipo producción,18826.916,2023-01-01 01:00:00+00:00,2023,1,1,1
3,10195,Generación medida total tipo producción,18157.879,2023-01-01 02:00:00+00:00,2023,1,1,2
4,10195,Generación medida total tipo producción,17020.369,2023-01-01 03:00:00+00:00,2023,1,1,3


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164211 entries, 0 to 164210
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype              
---  ------    --------------   -----              
 0   id        164211 non-null  int64              
 1   name      164211 non-null  category           
 2   value     164211 non-null  float64            
 3   datetime  164211 non-null  datetime64[ns, UTC]
 4   year      164211 non-null  int32              
 5   month     164211 non-null  int32              
 6   day       164211 non-null  int32              
 7   hour      164211 non-null  int32              
dtypes: category(1), datetime64[ns, UTC](1), float64(1), int32(4), int64(1)
memory usage: 6.4 MB


In [71]:
fecha_corte = pd.Timestamp('2023-01-10 12:00:00+00:00')

In [72]:
dummy = df[df['datetime'] < fecha_corte]


In [73]:
dummy2 = dummy.drop(dummy[dummy['id']== 10195].index)

In [102]:
dummy2['date'] = dummy2['datetime'].dt.date

In [80]:
# Generar un gráfico de área acumulada
alt.Chart(dummy2).mark_area(interpolate='step').encode(
    x=alt.X('datetime:T', title='Hora del día'),  # Eje X: Hora del día
    y=alt.Y('sum(value):Q', title='Potencia eléctrica (MW)'),  # Suma de potencia
    color=alt.Color('name:N', title='Tecnología')  # Diferenciar por tecnología
).properties(
    title='Generación eléctrica por tecnología',
    width=800,
    height=400
)

alt.Chart(...)

In [97]:
pn.extension('vega')

In [98]:
# https://towardsdatascience.com/how-to-build-a-time-series-dashboard-in-python-with-panel-altair-and-a-jupyter-notebook-c0ed40f02289
date_slider = pn.widgets.DateSlider(name='Date Slider', start=dt.datetime(2023, 1, 1), end=dt.datetime(2023, 12, 31), value=dt.datetime(2023, 1, 1))

date_slider

BokehModel(combine_events=True, render_bundle={'docs_json': {'c5bcff7d-783e-4c04-8ffa-1d6864d1da85': {'version…

In [104]:
@pn.depends(date_slider.param.value)
def get_plot(date):
     # Load and format the data
     df = dummy2 # define df
     ##df[‘date’] = pd.to_datetime(df[‘date’])
     # create date filter using values from the range slider
     # store the first and last date range slider value in a var
     start_date = date_slider.value
     # create filter mask for the dataframe
     mask = (df['date'] == start_date)
     df = df.loc[mask] # filter the dataframe
     # create the Altair chart object
     chart = alt.Chart(df).mark_area(interpolate='step').encode(
               x=alt.X('datetime:T', title='Hora del día'),  # Eje X: Hora del día
               y=alt.Y('sum(value):Q', title='Potencia eléctrica (MW)'),  # Suma de potencia
               color=alt.Color('name:N', title='Tecnología')  # Diferenciar por tecnología
               ).properties(
               title='Generación eléctrica por tecnología',
               width=800,
                height=400
)
     return chart

In [105]:
title = '### Stock Price Dashboard'
subtitle = 'This dashboard allows you to select a company and date range to see stock prices.'

dashboard = pn.Row(pn.Column(title, subtitle, date_slider),
get_plot # our draw chart function!
)

/usuarios/jos/.conda/envs/jos_prueba/lib/python3.8/site-packages/panel/viewable.py:298: ParamFutureWarning: Parameter 'object' on <class 'panel.pane.base.PaneBase'> is being given a valid parameter reference <function get_plot at 0x7f175cd044c0> but is implicitly allow_refs=False. In future allow_refs will be enabled by default and the reference <function get_plot at 0x7f175cd044c0> will be resolved to its underlying value alt.Chart(...). Please explicitly set allow_ref on the Parameter definition to declare whether references should be resolved or not.
  super().__init__(**params)


In [106]:
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'324b505b-94c2-4987-808c-8dc5046175b3': {'version…

In [86]:
# Generar el gráfico de área acumulada
base = alt.Chart(dummy2).mark_area(interpolate='step').encode(
    x=alt.X('datetime:T', title='Hora del día'),  # Eje X: Hora del día
    y=alt.Y('sum(value):Q', title='Potencia eléctrica (MW)'),  # Suma de potencia
    color=alt.Color('name:N', title='Tecnología')  # Diferenciar por tecnología
).properties(
    title='Generación eléctrica por tecnología',
    width=800,
    height=400
).add_params(anio_select, mes_select, dia_select
).transform_filter(
    (alt.datum.anio == anio_select.anio) & 
    (alt.datum.mes == mes_select.mes) & 
    (alt.datum.dia == dia_select.dia)
)

In [87]:
base

alt.Chart(...)

In [ ]:
df = df.pivot(index='datetime', columns='name', values='value').reset_index()

In [ ]:
df

In [ ]:
alt.Chart(df).mark_line().encode(
  x='datetime:T',
  y='Precio medio horario componente banda secundaria:Q'
)

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(df).mark_line().encode(
  x='datetime:T',
  y='value:Q',
  color='name:N'
)

In [ ]:
alt.Chart(df).mark_bar().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N'
)

In [ ]:
selection = alt.selection_point(fields=['name'], bind='legend')

In [ ]:
# Plotting
chart.mark_line().encode(
    x='datetime:T',
    y='value:Q',
    tooltip=['datetime:T', 'value:Q']
).properties(
    title='Hourly Average Price Over Time',
    width=800,
    height=400
).interactive()





In [ ]:
chart.show

In [ ]:
# Crear el gráfico apilado
chart.mark_area().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N',
    tooltip=['datetime:T', 'name:N', 'value:Q']
).properties(
    title='Evolución del Precio Horario de la Electricidad en 2023 por Componentes',
    width=800,
    height=400
).interactive()


In [ ]:
# Crear el gráfico apilado
alt.Chart(df).mark_area().encode(
    x='datetime:T',
    y='value:Q',
    color='name:N',
    tooltip=['datetime:T', 'name:N', 'value:Q']
).properties(
    title='Evolución del Precio Horario de la Electricidad en 2023 por Componentes',
    width=800,
    height=400
).interactive()